# FMCA Gaussian process learning

### first import modules

In [ ]:
# import seems necessary to not crash matplotlib
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
import matplotlib.pyplot as plt
import numpy as np
from numpy import linalg as la
import time
import FMCA

### generate data points and visualize them
(We force NumPy to use column major arrays for performance reasons)

In [ ]:
dim = 1
N = 100
x = 2 * np.array(np.random.rand(1, N), order='F') - 1
y = np.sin(10 * x) * np.cos(20 * x) + (x+1)**2 + 0.05 * np.random.randn(1, N)
plt.figure(figsize=(15,5))
plt.plot(x, y, 'b.')
plt.show()

### compute a lowrank approximation to the kernel matrix
(We force NumPy to use column major arrays for performance reasons)

In [ ]:
start = time.time()
cov = FMCA.CovarianceKernel("gaussian", 0.01)
Chol = FMCA.PivotedCholesky()
Chol.computeFullPiv(cov, x, 1e-2)
L = Chol.matrixL()
stop = time.time()
print('elapsed time lowrank: ', stop - start, 'sec.')

### compute representer using the pseudo inverse of the Cholesky factor
(Basis elements can be retrieved by inverse transforming unit vectors)

In [ ]:
LTL = L.transpose() @ L
LTy = (y @ L).transpose()
z = np.linalg.solve(LTL, LTy)
mu = L @ np.linalg.solve(LTL, z)

### visualize rerpresenter

In [ ]:
M = 2000;
xeval = np.reshape(np.linspace(-1, 1, M), (1, M), order='F')
Keval = cov.eval(xeval, x)
# compute posterior covariance
LTKevalT = (Keval @ L).transpose()
z = np.linalg.solve(LTL, LTKevalT)
S = LTKevalT.transpose() @ np.linalg.solve(LTL, z)
post_var = np.reshape((cov.eval(xeval, xeval)-S).diagonal(), (1, M), order='F')
std = np.sqrt(np.maximum(post_var, 0))
mu_eval = (Keval @ mu).transpose();
# plot everything
plt.figure(figsize=(15,5))
plt.fill_between(xeval[0,:], (mu_eval-3*std)[0,:], (mu_eval+3*std)[0,:], color='green', alpha=0.3)
plt.plot(x, y, 'bo', markersize=4)
plt.plot(xeval[0,:], mu_eval[0,:],'r')
plt.show()